In [2]:
import nltk 
from nltk import PCFG
grammar=PCFG.fromstring("""
S -> NP VP [0.9]
S -> VP [0.1]
VP -> V NP [0.5]
VP -> V [0.5]
NP -> Det N [0.3]
NP -> N [0.7]
N -> 'cat' [0.2]
N -> 'book' [0.2]
N -> 'bird' [0.2]
N -> 'dog' [0.4]
V -> 'read' [0.1]
V -> 'chased' [0.6]
V -> 'ate' [0.3]
Det -> 'the' [0.5]
Det -> 'a' [0.5]
""")

def cyk(pcfg,sentence):
    n=len(sentence)
    table=[[[] for _ in range(n)]for _ in range(n)]
    for i in range(n):
        for prod in pcfg.productions(rhs=sentence[i]):
            table[i][i]=[(prod.lhs(),prod.prob())]
    for length in range(2,n+1):
        for i in range(n-length+1):
            j=i+length-1
            for k in range(i,j):
                for prod in pcfg.productions():
                    for left,left_prob in table[i][k]:
                        for right,right_prob in table[k+1][j]:
                            if prod.rhs()==(left,right):
                                prob=left_prob*right_prob*prod.prob()
                                table[i][j].append((prod.lhs(),prob))
    print(table)
    return table[0][n-1]                 
sent="the cat chased the dog"
prob=cyk(grammar,sent.lower().split())
prob

[[[(Det, 0.5)], [(NP, 0.03)], [], [], [(S, 0.00048599999999999994)]], [[], [(N, 0.2)], [], [], []], [[], [], [(V, 0.6)], [], [(VP, 0.018)]], [[], [], [], [(Det, 0.5)], [(NP, 0.06)]], [[], [], [], [], [(N, 0.4)]]]


[(S, 0.00048599999999999994)]

In [4]:
parser=nltk.ViterbiParser(grammar)
token=nltk.word_tokenize(sent)
trees=list(parser.parse(token))
for tree in trees:
    print(tree)
    tree.pretty_print()

(S
  (NP (Det the) (N cat))
  (VP (V chased) (NP (Det the) (N dog)))) (p=0.000486)
              S               
      ________|_____           
     |              VP        
     |         _____|___       
     NP       |         NP    
  ___|___     |      ___|___   
Det      N    V    Det      N 
 |       |    |     |       |  
the     cat chased the     dog

